# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [272]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 


Поддерживаются две операции - сложение и умножение. Применить можно так:

In [273]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)


22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [274]:
# ваш код
from math import log
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number


@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)

    def __abs__(self) -> "Dual":
        return Dual(abs(self.value), abs(self.d))

    def __invert__(self) -> "Dual":
        return Dual(~self.value, self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(float(other) - self.value, self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d)/(o_value*o_value))
            case Number():
                return Dual(self.value/float(other), self.d / float(other) )

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value / self.value, (o_d * self.value - o_value * self.d)/(self.value*self.value))
            case Number():
                return Dual(float(other) / self.value, float(other) * (-self.d)/(self.value*self.value))

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, o_value * (self.value ** (o_value - 1)) * self.d + (self.value ** o_value) * o_d * log(self.value))
            case Number():
                return Dual(self.value ** float(other), float(other) * (self.d * self.value ** (float(other) - 1)))

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

In [275]:
from random import randint, random, uniform

def test_sum_neg():
    for tmp_v, tmp_d in test_pair():
        sum_v = Dual(tmp_v, tmp_d) + (-Dual(tmp_d, tmp_v))
        exp_v = Dual(tmp_v - tmp_d, tmp_d - tmp_v)
        if sum_v.value != exp_v.value or sum_v.d != exp_v.d:
            print(False)

def test_mul_div():
    for tmp_v, tmp_d in test_pair():
        if tmp_v == 0 or tmp_d == 0:
            continue
        mul_v = Dual(tmp_v, tmp_d) * Dual(tmp_d, tmp_v)
        div_v = Dual(tmp_v, tmp_d) / (Dual(1.0, 0.0) / Dual(tmp_d, tmp_v))
        if abs(mul_v.value - div_v.value) > 1e-8 or abs(mul_v.d - div_v.d) > 1e-8:
            print(False)

def test_mul_pow():
    for tmp_v, tmp_d in test_pair():
        n = randint(1, 5)

        mul_v = Dual(tmp_v, tmp_d)
        for i in range(n-1):
            mul_v = mul_v * Dual(tmp_v, tmp_d)
        
        pow_v = Dual(tmp_v, tmp_d) ** n
        if abs(mul_v.value - pow_v.value) > 1e-8 or abs(mul_v.d - pow_v.d) > 1e-8:
            print(False)
        

def test_pair():
    return tuple([((random() + 0.5) * 3.0, (random() + 0.5) * 3.0) for _ in range(15)])

In [276]:
test_sum_neg()
test_mul_div()
test_mul_pow()

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [277]:
# ваш код
import math

def exp(f : Union["Dual", Number]):
    match f:
        case Dual(f_value, f_d):
            return Dual(math.exp(f_value), f_d*math.exp(f_value))
        case Number():
            return math.exp(f)

def cos(f : Union["Dual", Number]):
    match f:
        case Dual(f_value, f_d):
            return Dual(math.cos(f_value), -f_d*math.sin(f_value))
        case Number():
            return math.cos(f)

def sin(f : Union["Dual", Number]):
    match f:
        case Dual(f_value, f_d):
            return Dual(math.sin(f_value), f_d*math.cos(f_value))
        case Number():
            return math.sin(f)

def log(f : Union["Dual", Number]):
    match f:
        case Dual(f_value, f_d):
            if f_value <= 0:
                raise ValueError("f_value <= 0!")
            return Dual(math.log(f_value), f_d/f_value)
        case Number():
            if f <= 0:
                raise ValueError("f <= 0!")
            return math.log(f)

def test_exp_log():
    for tmp_v, tmp_d in test_pair():
        test_v = Dual(tmp_v, tmp_d)

        if abs((log(exp(test_v)) - test_v).value) > 1e-8 or abs((log(exp(test_v)) - test_v).d) > 1e-8:
            print(False)

def test_sin_cos():
    for tmp_v, tmp_d in test_pair():
        test_v = Dual(tmp_v, tmp_d)
        if abs((cos(test_v)**2 + sin(test_v)**2).value) - 1 > 1e-8 or abs((cos(test_v)**2 + sin(test_v)**2).d) > 1e-8:
            print(False)

test_exp_log()
test_sin_cos()

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [278]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

print(derivative(f, 2.))
f_diff = diff(f)
print(f_diff(2))

22.0
22.0


In [279]:
# ваш код
from numpy import isclose
def f_1(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f_2(x: float) -> float:
    return 5 / x * x - 2*x + cos(x)/log(x)

def f_3(x: float) -> float:
    return 15 / (x**2) + 3 * x + 15

def f_4(x: float) -> float:
    return 1/cos(x) - 15 * sin(15 * x)

def f_5(x: float) -> float:
    return sin(x) - log(x)

def f_6(x: float) -> float:
    return exp(15*x) - cos(4 * x + 5) + x ** (15 * x)

def f_7(x: float) -> float:
    return 2 * log(x ** 7) + cos( exp ( x))


for i in range(10):
    x = uniform(-10, 10)
    f_diff = diff(f_1)
    if not isclose(f_diff(x), derivative(f_1, x, dx=1e-8)):
        print(False)
    f_diff = diff(f_2)
    if not isclose(f_diff(abs(x)), derivative(f_2, abs(x), dx=1e-8)):
        print(False)
    f_diff = diff(f_3)
    if not isclose(f_diff(x), derivative(f_3, x, dx=1e-8)):
        print(False)
    f_diff = diff(f_4)
    if not isclose(f_diff(x), derivative(f_4, x, dx=1e-8)):
        print(False)
    f_diff = diff(f_5)
    if not isclose(f_diff(abs(x)), derivative(f_5, abs(x), dx=1e-8)):
        print(False)
    f_diff = diff(f_6)
    if not isclose(f_diff(abs(x)), derivative(f_6, abs(x), dx=1e-8)):
        print(False)
    f_diff = diff(f_7)
    if not isclose(f_diff(abs(x)), derivative(f_7, abs(x), dx=1e-8)):
        print(False)


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [280]:
# ваш код
def diff(f: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: f(Dual(x, 1.0)).d 

def diff_m_f(f):
    def diff_f(**args):
        ds = []
        for arg in args:
            tmp_args = args.copy()
            tmp_args[arg] = Dual(args[arg], 1.0)
            dual = f(**tmp_args)
            ds.append(dual.d)
        return ds
    return diff_f

In [281]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_diff = diff_m_f(f)

f_diff(x=10, y=10, z=10)

[10.0, 5.0, 1.0]